![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/16.GPT2_Transformer_In_Spark_NLP.ipynb)

# GPT2Transformer: OpenAI Text-To-Text Transformer

![image.png](https://miro.medium.com/max/1400/1*UZTblFMn8oWfmqBMQhUOdA.jpeg)

[GPT-2](https://openai.com/blog/better-language-models/)  displays a broad set of capabilities, including the ability to generate conditional synthetic text samples of unprecedented quality, where the model is primed with an input and it generates a lengthy continuation.

Pretrained models can be loaded with `pretrained()` of the companion object:

**GPU MODE** is highly reccomended, since this is a very large model.

## Colab Setup

In [ ]:
! pip install -q pyspark==3.3.0 spark-nlp==4.2.4

In [ ]:
import sparknlp

from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

**Make sure to Enable GPU Mode and High RAM**

In [ ]:
# Comment out this line  and uncomment the next one to enable GPU mode and High RAM
# 
spark = sparknlp.start()

# spark = sparknlp.start(gpu=True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

**GPT2 Models In Spark NLP**

*   `gpt2`
*   `gpt2_medium`
*   `gpt2_distilled`
*   `gpt2_large`

The default model is `"gpt2"`, if no name is provided. For available pretrained models please see the [Spark NLP Models Hub](https://nlp.johnsnowlabs.com/models?q=gpt2)



## GPT2 Pipeline 

Now, let's create a Spark NLP Pipeline with `gpt2_medium` model and check the results.

In [5]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")
    
gpt2 = GPT2Transformer.pretrained("gpt2_medium") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(50) \
    .setMinOutputLength(25) \
    .setOutputCol("generation")
    
data = spark.createDataFrame([["My name is Leonardo."]]).toDF("text")

pipeline = Pipeline().setStages([documentAssembler,gpt2]).fit(data)

result = pipeline.transform(data)
result.select("generation.result").show(truncate=False)

gpt2_medium download started this may take some time.
Approximate size to download 1.2 GB
[OK!]
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                           |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[ My name is Leonardo. I'm a student at the University of California, Berkeley. I've been studying computer science for the past two years. I have a PhD in computer scienc

We can display the documentation of all params with their optionally default values and user-supplied values by `explainParams()` function

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: False)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for repetition penalty. 1.0 means no penalty. See `this paper <https://arx

Let's use model with more sentences and set `.setDoSample()` parameter as True, this parameter is used for whether or not to use sampling; use greedy decoding otherwise, by default False. <br/>
Also, we use `.setTopK()` parameter for the number of highest probability vocabulary tokens to keep for top-k-filtering, by default 50.

In [ ]:
sample_texts= [[1, "Mey name is  Leonardo"], 
               [2, "My name is Leonardo and I come from Rome."],
               [3, "My name is"], 
               [4, "What is the difference between diesel and petrol?"]]

sample_df= spark.createDataFrame(sample_texts).toDF("id", "text")

In [ ]:
gpt2.setMaxOutputLength(50).setMinOutputLength(25).setDoSample(True).setTopK(20)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                                                                                 |
+---+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Mey name is  Leonardo. He was born in 1884.\n\nThe other name for my family was the Tien, which means to die. It's the same as being bo

### Changing the Transformer's task

Now, we change the task of Transformer. We can verify some informations to GPT-2 by setting `.setTask()` parameter as **"Is it true that"**. <br/>
We give a text to the model by setting `setTask("Is it true that")` and model adds the "Is it true that" expression at the beginning of the sentence and generates sentences.

In [ ]:
gpt2.setTask("Is it true that").setMaxOutputLength(50)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
sample_text= [[1, "Donald Trump is rich?"],
              [2, "Pink Floyd is rock band?"]]

sample_df= spark.createDataFrame(sample_text).toDF("id", "text")

result = pipeline.transform(sample_df)
result.select("id", "generation.result").show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |result                                                                                                                                                                                                                              |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1  |[ Is it true that Donald Trump is rich?\n\nDonald Trump: Well, I think the people that are doing really extremely well right now are the people on the lower rungs of the income scale. The reason I say that is because I'm making]|
|2  |[ Is it true that Pink Floyd is rock band? I mean they'

# Some Helper Methods for generating and printing generation results

In [7]:
from typing import List
def prompts_to_spark_df(prompts,spark=spark): 
  text = [[i, prompt] for i,prompt in enumerate(prompts)]
  return spark.createDataFrame(text).toDF("id", "text")  

def generate_with_gpt2(gpt_pipe, prompts,task='generate', print=True):
  if isinstance(prompts,str):
    df = prompts_to_spark_df([prompts])
  elif isinstance(prompts,List):
    df = prompts_to_spark_df(prompts)
  else : 
    raise ValueError(f"Invalid Type = {type(prompts)} please pass a str or list of str for prompts parameter ")
  df = pipeline.transform(df)
  df = df.select("id",'text', "generation.result").toPandas()
  
  if print:
    print_generation_results(df)
  return df 

def print_generation_results(df):
  for idx, row in df.iterrows():
    print(f'Example {idx}: {200*"_"}')
    print(row.result[0])
    print('\n')


# Explore GPT Parameters Play with Paramns

### Sampling Methods


Sampling means we **randomly** draw from a distribution of words. 
The probability distribution is conditioned on all previous tokens in a text to generate the next token. 

By default the distribution contains all words in the vocabulary of GPT2, where many candidates are incorrect to generate.

There are two methods of reshaping and drawing from those distributions : 

1. **Top-K Sampling** Take the k most likely words from the original distribution. Redistribute probability mass among those k words and draw according to the new probabilities.

2. **Top-P Nucleus sampling**  Take smallest possible set of N words, which  together have a probability of p. Redistribute probability mass among those N words and draw according to the new probabilities.



Additionally, both methods can be tweaked ith the following parameters : 

- **temperature** : Parameter of the softmax function which affect the distrubtion computed by the model. The closer we are to 0, the more deterministic the probability will become, distribution tails will become slimmer and outlier word probabilites are more close to 0. Temperature values closer values to 1 make tails of probability fatter which makes outliers more probable and generic results less probable. 


These parameters are shared by all method : 
- **ignoreTokenIds**: A list of token ids which are ignored in the decoder's output (default: [])
- **noRepeatNgramSize**: If set to int > 0, all ngrams of that size can only occur once 
- **repetitionPenalty**: The parameter for repetition penalty. 1.0 means no penalty.  https://arxiv.org/pdf/1909.05858.pdf> 
- **task**:  Transformer's task, e.g. 'is it true that'> (default: , current: generate)

In [ ]:
print(gpt2.explainParams())

batchSize: Size of every batch (default: 4)
configProtoBytes: ConfigProto from tensorflow, serialized into byte array. Get with config_proto.SerializeToString() (undefined)
doSample: Whether or not to use sampling; use greedy decoding otherwise (default: False, current: True)
ignoreTokenIds: A list of token ids which are ignored in the decoder's output (default: [])
inputCols: previous annotations columns, if renamed (current: ['documents'])
lazyAnnotator: Whether this AnnotatorModel acts as lazy in RecursivePipelines (default: False)
maxOutputLength: Maximum length of output text (default: 20, current: 50)
minOutputLength: Minimum length of the sequence to be generated (default: 0, current: 25)
noRepeatNgramSize: If set to int > 0, all ngrams of that size can only occur once (default: 0, current: 3)
outputCol: output annotation column. can be left default. (current: generation)
repetitionPenalty: The parameter for repetition penalty. 1.0 means no penalty. See `this paper <https://arxi

### Play with temperature 
Set Temperature higher to make GPT more random/creative and text less coherent
Temperature > 0  and Temperature <=1
You must set `gpt2.setDoSample(True)` to have non-deterministic results

In [ ]:
text = """Hello my name is GPT2, I love to """
data = [text, text,text,text,text ]
gpt2.setMaxOutputLength(200)

GPT2TRANSFORMER_729a8e69e804

In [ ]:
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to irl, but my boobs don`t look like other girls?

I`m going to answer them here: yes they do!

Hello My Name Is GPT is a game where you wear a bra. And you have a choice whether or not to play with other girls and do crazy things like take a blowjob, or try taking one and masturbating it off.

You are able to customize your game by choosing whether or no you want the bra to cover your nipples, and what it looks like in the girls room. In the beginning I was not so sure about it - but after playing for some time now I think it`s a wonderful game. I would recommend it to anyone who does not like bra wearing, and wants to see what you get up to. You can also customize your character a little bit: you can change your hair color, your 

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.75)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! Reply Delete
I've been looking for a couple days for an image editor that I can edit images from my computer without having to download software. My main computer has an older version of Photoshop which is very slow and I'm not sure if I can use it on a modern computer. I've been using Lightroom and Photoshop with no luck until I discovered this application. It's a wonderful program but I'm looking for more. Reply Delete


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I've never been able to download it but 

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.5)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

Hello my name are GPT3, I am a girl, I'm a girl and I'm gonna !!!


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!


Hello my name.


I am GPT3, I am a girl I love !!!

I love !!!!


Hi my name, I'm GPT4, I like !!!


Example 2: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ____ and I am a ____?

GPT2: Yes, 

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.25)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline,data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!!

I have been playing with GPT since the beginning of the year and I am very happy with it. I have been using it for about 1 month now and I have not had any problems with it and I think it is a great product. I am also using it to make my own games. I would recommend this product to anyone who wants to make their own games and I would also recommend it to anyone that wants to learn how things work.
 I have used the GPT 2 for about a month now. I was very happy to see that the GPH is working well. I did not have any problems at all with the GPG. I also did not experience any issues with the audio or the screen. I do not know if it is because I am using the GPS or if it has something to make the GPP work better. I will be usin

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to !!! But is it true? Can I install some packages of Debian 8 by a community software to work my Ubuntu/Mac systems from here? It is, this site also offer, in general, several of its projects under license GNU GPLv3. Are you in trouble as GPT3 version and all software with similar or older copyright will be rejected at the last because of different code? So in addition to install Linux software of its Debian equivalent as I need only do something more serious I recommend Linux Debian to do my need because we could build new life without software restrictions by use of some third party toolchain of free/open source Software . The question on here might be very tricky when my main motivation from here is to keep with our freedom wit

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


In [ ]:
gpt2.setTemperature(0.0001)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, data ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Hello my name is GPT2, I love to ________________, HiGTPiRzCq4Nc7H3R7c1jdPtPX9P5p4x
It does however work just okay so its definitely more like how an eee can feel with gppa . Not great yet for someone to install from usb though if only one guy or so gets hold in him!So that might ormay explain, just so no surprises:My eemup does appear better however...it appears slightly weaker dueto what a bunch nuke said that we already already found here as stated a previous week..If they will keep talking then more testing are surely being ordered soon enoughI've made more sure, thanks very kindly people are interested :),Hope everything go very right this new development cycle in linux . This project also deserves our applause if i'll continue its working !It just got ver

,id,text,result
0,0,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
1,1,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
2,2,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
3,3,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."
4,4,"Hello my name is GPT2, I love to","[ Is it true that Hello my name is GPT2, I lov..."


# GPT2 Use Cases

These are some examples to give you an idea of what GPT2 is capable of 

## GPT for Programming Code

### GPT and Scala

In [ ]:
# Hello world in Scala
scala_code = """
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, scala_code ,task='generation', print=True)



Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
object Hello {
    def main(args: Array[String]) = {
        println("Hello, world")
    }
}
Object<String, Hello> object
Hello,
Object.class:3:2: warning: cannot convert String to int using equals
def main( args : Array[ String ], main : IO(Unit) => Unit) = { val hello = object.toString("hello, world");
println("Hello,");
} Hello,
Hello.class
Object[String, String],
The following line of code throws a syntax error:
object.toLongArray:4:5: error: cannot do long array conversion in an object literal
A very similar syntax error appears in the code for the following class:
class MyObject { def toLongObject(): Long = { println( Long.fromNumber(2)).toString(); } }
The code below is equivalent to the code above:
import java.math.BigDecimal import java.lang.Math; imp

,id,text,result
0,0,\nobject Hello {\n def main(args: Array[Str...,[ Is it true that \nobject Hello {\n def ma...


### GPT and Python

In [ ]:
# adding 3 numbers in python
py_code = """
def my_func(a,b,c):
  return a+b+c
"""
gpt2.setMaxOutputLength(1000)

gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, py_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def my_func(a,b,c):
  return a+b+c

end

The function definition

defmy_func f =

if !is_a:

raise TypeError("Failed to define my_fn!")

return

self .my_function(f)





,id,text,result
0,0,"\ndef my_func(a,b,c):\n return a+b+c\n","[ Is it true that \ndef my_func(a,b,c):\n ret..."


In [ ]:
# breadth-first search 
bfs_python_code = """
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0) 
    print (s, end = " ") 

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
"""


gpt2.setMaxOutputLength(1000)
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, bfs_python_code ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
def bfs(visited, graph, node):
  visited.append(node)
  queue.append(node)

  while queue:
    s = queue.pop(0) 
    print (s, end = " ") 

    for neighbour in graph[s]:
      if neighbour not in visited:
        visited.append(neighbour)
        queue.append(neighbour)
end

  if queue: 

  print "  "
 
end

(defn node-get-children [path, children] (let [path (map #(string path) "children") ] (fn [p] ( if child (map child path) p))))

((fn [path child] (map :tree path) parents))

)

NodeGet: A simple method to get a node from a tree.

The methods in node-child are not really methods, only functions, and thus they can be used together in other functions of different trees. You can pass an index of your tree or a string to the children function, to get the elem

,id,text,result
0,0,"\ndef bfs(visited, graph, node):\n visited.ap...","[ Is it true that \ndef bfs(visited, graph, no..."


## Generate Product Recommendations

We can start a base list of top 10 anything and GPT2 will complete it for us!      
This is very useful to reccomendation and search engines.

### Top 10 Movie List

In [ ]:
movie_list = """My top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4."""

gpt2.setTemperature(1)
gpt2.setMaxOutputLength(200)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. The Green Mile 1
5. The Departed
6. Saving Private Ryan
7. Braveheart
8. Gladiator 1
9. Raiders of the Lost Ark 1
10. The Passion of the Christ 1. The first Matrix movie is considered to be the most popular movie of all time; the second is regarded as less popular; and the third and last Matrix movie remains a bit of a mystery. I have no choice but to choose it as the number one. But how do you decide which one is the best? In order to answer this, I've been researching and analyzing every movie in which Matrix's character Neo (David Lynch/Robert Altman, the director of the film) had an important role. I then compiled my own list of movies that I felt fit that description, and ranked the

,id,text,result
0,0,My top 10 movie list : \n1. The Matrix 1\n2. T...,[ Is it true that My top 10 movie list : \n1. ...


How **not** to generate text with GPT2

Since every new word is conditioned on every previous **character**, things like **white spaces** and **new lines** can skew the sampling distribution into an unintended direction

In [ ]:
# NOTE! We have malformed the input text and added a newline. 
# This will Confuse GPT2, it might think the list is over and probability distribution changes accordingly
movie_list = """My Top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. 

"""
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 movie list : 
1. The Matrix 1
2. The Terminator 1
3. Scarface
4. 


I was also shocked at the number two place for a film on this list.  I was expecting the number one film to be something like  The Matrix 2.  .  If this were a list of Top 10 movies, they would surely be one of the few to take this spot. . 
My Top 10 film list :
1) The Matrix
2) 
3) Terminator 2: Judgment Day
4)
5)  (5th best movie)  
6) The Terminator 2
7) (6th best)   (8th best film)   8) The Dark Knight
9) (10)
10) 

 A number of the movies above have been in my top 10 for a number of years. 
I thought of the




,id,text,result
0,0,My Top 10 movie list : \n1. The Matrix 1\n2. T...,[ Is it true that My Top 10 movie list : \n1. ...


In [ ]:
movie_list = """My Top 10 worst movie list : 
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, movie_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 worst movie list : 
1. The Matrix 4
2. Jack and Jill
3. Super Mario Bros. (1993)
4.  The Exorcist 3 (1992)
5.   A Nightmare on Elm Street (1984)
6.    Memento (1992)"
7.      Star Wars (1977)
8.      The Day of the Triffids (1978)
9.        The Exorcists  2 (1992), 1, 5 (1989)
10.  A Nightmare On Elm Street II (1984),  13 (1977),    1 (1989),    5 (1982)
My top 10 worst movies list :
-     A Nightmare on  Elm Street (1985)  -  
-   The Exor cists (1984).   - 
- Star




,id,text,result
0,0,My Top 10 worst movie list : \n1. The Matrix 4...,[ Is it true that My Top 10 worst movie list :...


### Top 10 Game List

In [ ]:
game_list = """My Top 10 video games of all time list : 
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 video games of all time list : 
1. Half-Life 2
2. Super Smash Bros Melee
3. Portal
4.  The Legend of Zelda: Link's Awakening
5. Metroid Prime: Federation Force
6. Star Wars: The Old Republic
7. The Sims - 4
8. Grand Theft Auto III
9. Minecraft
10. Diablo 3
I guess it can't be all bad.
I know a lot of people like video games so let's give each game its own article!
A lot of games are more important than other games, but what can people expect of their favorite games?  I have put together a list of the 10 best video games.     
If you have any questions about any of these or think I forgot something that we missed, let me know in the comments below!




,id,text,result
0,0,My Top 10 video games of all time list : \n1. ...,[ Is it true that My Top 10 video games of all...


### Top 10 Book List

In [ ]:
game_list = """My Top 10 books of all time list : 
1. 1964
2. Moby Dick
3. robinson crusoe
4. """
gpt2.setTemperature(1)
gpt2.setDoSample(True)
generate_with_gpt2(pipeline, game_list ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that My Top 10 books of all time list : 
1. 1964
2. Moby Dick
3. robinson crusoe
4.  The Last Man On Earth
5. The Adventures Of Tintin
6. The Jungle Book
7.  Lord of the Rings  8. The Lord of The Rings  
9. The Wizard of Oz 
10.  The Little Prince 




,id,text,result
0,0,My Top 10 books of all time list : \n1. 1964\n...,[ Is it true that My Top 10 books of all time ...


## SEO Text / Content Writing / Copy Writing
Lets generate some texts for our new imaginary shopify store!
We will sell some hoodies, soap and beard products.

Based on some base templates, we can generate high quality marketing/seo texts

In [ ]:
# from https://www.shopify.com/blog/8211159-9-simple-ways-to-write-product-descriptions-that-sell 

# Lets define some base texts, from which we will generate new texts

hoodie = """Can’t stop buying plants? Unbeleafable. Don’t worry—us too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie! """

soap  = """Made with real pine extract, this all-star bar is as tough as a freshly cut bat. 
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent."""

beard = """ Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control, """
gpt2.setTemperature(1)
gpt2.setDoSample(True)

GPT2TRANSFORMER_729a8e69e804

#### Marketing Texts for Hoodies

In [ ]:
# Lets generate some marketing texts for our hoodies
generate_with_gpt2(pipeline, [hoodie,hoodie,hoodie,hoodie,hoodie,hoodie] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Cant stop buying plants? Unbeleafable. Dont worryus too! Cover yourself in your favourite obsession in our NEW I Love Plants Oodie!  I was in a panic at the beginning of the week and couldn't figure out what I was doing, so I did a bit of reading... The first book in this list is an amazing one... My favourite thing about this book?  It's a collection of illustrations of the plants that have always been part of me... the tree, the flower and the grass.  The illustrations are so beautiful and I think it was quite fun to do!   Each of the pictures is so detailed, and I loved that all my favourite plants, like this one, had their own names as well! 
I love the colours in the book... blue, pink, white, yellow, purple, green, and purple.    It's really easy to pick 

,id,text,result
0,0,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
1,1,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
2,2,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
3,3,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
4,4,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...
5,5,Can’t stop buying plants? Unbeleafable. Don’t ...,[ Is it true that Cant stop buying plants? Unb...


#### Marketing Texts for Soap

In [ ]:
# Lets generate some marketing texts for our soap
generate_with_gpt2(pipeline, [soap,soap,soap,soap,soap,soap] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat. 
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent. A perfect base for cocktails, this is your man's bar.


Example 1: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that Made with real pine extract, this all-star bar is as tough as a freshly cut bat. 
A true MVP of the shower, this heavy-hitter knocks out grime with its gritty composition and ultra-manly, woodsy scent.  
This is one bar where I believe we're all looking to get back to our roots.


Ex

,id,text,result
0,0,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
1,1,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
2,2,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
3,3,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
4,4,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."
5,5,"Made with real pine extract, this all-star bar...","[ Is it true that Made with real pine extract,..."


#### Marketing Texts for beard products

In [ ]:
# Lets generate some marketing texts for our beard products
generate_with_gpt2(pipeline, [beard,beard,beard,beard,beard,beard] ,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that  Whatever your style is, Beardbrand Styling Balm is versatile enough to handle it.
Designed to work with all hair types, it provides enough hold to keep thick, curly hair under control,  but doesn't pack in heavy oil to break down hair. 
This is our favorite brand and we highly recommend it.  If you're interested in getting it for less, check out the below: Beardbrand (BEST STYLE, HIGH SPEED)  ( $7.50 ) BEST SPIRAL LABEL:
This is an amazing shampoo with a unique and unique formula.     It gives your hair the  perfect  shine and shine to keep it  clean and shine .  For goes, this one has a  special inclusions for your go to products, , and is  also  compatible with the newest skincare brands, such as


Example 1: _________________________________________________

,id,text,result
0,0,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
1,1,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
2,2,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
3,3,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
4,4,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."
5,5,"Whatever your style is, Beardbrand Styling Ba...","[ Is it true that Whatever your style is, Bea..."


## Song Lyrics

#### Last Christmas

In [ ]:
last_christmas = """
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, last_christmas,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 Is it true that 
Last Christmas, I gave you my heart
But the very next day, you gave it away
This year, to save me from tears
I'll give it to someone special
I can't imagine how much I would miss it
But I'll make you happy
It'll make us laugh and laugh a long time
Because we love you so so much
And we can't wait for the next gift day
Because this is the first Christmas
We'll never give up hope
And this Christmas, the first time,
I'm going away
We were so lucky
We're still so happy
But what can you do?
I just don't know
I really need someone to hold
All the hope I could give you
I need your help
We don't have enough money on hand
This Christmas, it's our last time
We won't be able to buy gifts
I want to buy everything you need but
No one will give me the time
And I don't want t

,id,text,result
0,0,"\nLast Christmas, I gave you my heart\nBut the...","[ Is it true that \nLast Christmas, I gave you..."


#### Fresh prince of GPT

In [8]:
bel_air = """
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, bel_air,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Now, this is a story all about how
My life got flipped-turned upside down
And I'd like to take a minute
Just sit right there
I'll tell you how I became the prince of a town called Bel-Air
And how I got my name
And what I did to get it
And why I'm the prince
And who I am
And where I'm from
And when I'm done
I want you to know
That I'm not a bad person
I'm just a bad kid
I don't deserve to be
I just want to be happy
I've got a lot of friends
I can't believe I'm still alive
I know I'm a bad guy
I was born to be a bad boy
I didn't grow up
I wasn't raised to be one
I never wanted to be anything
I wanted to do anything
But I was born bad
I had to be bad
And now I'm just bad
So I'm going to tell you
How I became a bad girl
And a bad prince
I got my first kiss
And my first love
And t

,id,text,result
0,0,"\nNow, this is a story all about how\nMy life ...","[ \nNow, this is a story all about how\nMy lif..."


#### GPT Jackson

In [9]:
thriller_lyrics = """
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
"""


gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, thriller_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
They told him, "Don't you ever come around here"
"Don't wanna see your face, you better disappear"
The fire's in their eyes and their words are really clear
So beat it, just beat it
You better run, you better do what you can
Don't wanna see no blood, don't be a macho man
You wanna be tough, better do what you can
So beat it, but you wanna be bad
Just beat it (beat it), beat it (beat it)
No one wants to be defeated
Showin' how funky and strong is your fight
It doesn't matter who's wrong or right
Just beat it (beat it)
Just beat it (beat it)
Don' wanna see blood, you gotta be tough
You gotta be strong, you got to be tough (beat)
You got to beat it out of me
You know you got a chance
You can't let me down
You're gonna be the one that's gonna take me down (beat, beat)
I'm gonna b

,id,text,result
0,0,"\nThey told him, ""Don't you ever come around h...","[ \nThey told him, ""Don't you ever come around..."


#### Real Slim GPT

In [10]:
real_slim_shady = """
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, real_slim_shady,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
May I have your attention, please?
May I have your attention, please?
Will the real Slim Shady please stand up?
I repeat, will the real Slim Shady please stand up?
We're gonna have a problem here?
Y'all act like you never seen a white person before
Jaws all on the floor like Pam, like Tommy just burst in the door
And started whoopin' her ass worse than before
They first were divorced, throwin' her over furniture (ah!)
It's the return of the, oh wait, no way, you're kidding
He didn't just say what I think he did, did he?
And Dr. Dre said-, nothing you idiots
Dr. Dre's dead, he's locked in my basement (haha)
I'm gonna have to get a new one, I'm gonna get a different one
I don't know what to do, I don't even know what I'm doing
I can't believe I'm still here, I can't even rememb

,id,text,result
0,0,"\nMay I have your attention, please?\nMay I ha...","[ \nMay I have your attention, please?\nMay I ..."


#### GPT Rap God Bot

In [11]:
rap_god_lyrics = """
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, rap_god_lyrics,task='generation', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Look, I was gonna go easy on you not to hurt your feelings
But I'm only going to get this one chance (six minutes-, six minutes-)
Something's wrong, I can feel it (six minutes, Slim Shady, you're on!)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means what I think it means, we're in trouble, big trouble
And if he is as bananas as you say, I'm not taking any chances
You are just what the doc ordered
I'm beginnin' to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now, who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rap-bot  (I'm a Rap-Bot, I rap, I Rap, I RAP)
I rap like I'm a robot
I don't care if I'm on the cover of the magazine, I don' know w

,id,text,result
0,0,"\nLook, I was gonna go easy on you not to hurt...","[ \nLook, I was gonna go easy on you not to hu..."


## Fiction Writing

#### Fantasy Stories

In [12]:
unicorn_story = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, 
previously unexplored valley, in the Andes Mountains. 
Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, unicorn_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 In a shocking finding, scientist discovered a herd of unicorns living in a remote, 
previously unexplored valley, in the Andes Mountains. 
Even more surprising to the researchers was the fact that the unicorns spoke perfect English.  The researchers were able to translate the words of the unicorn, and the researchers were even able to identify the unicore's name.  
The researchers believe that the unicorn's language is a combination of the languages of the Andean and the Tibetan languages.  The researchers believe the unicareans are descended from the ancient Andean people who lived in the region.  They believe that they are descended of the ancient Tibetan people who were the first to discover the Andeans. 




,id,text,result
0,0,"In a shocking finding, scientist discovered a ...","[ In a shocking finding, scientist discovered ..."


In [13]:
silly_story = """
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and 
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, silly_story,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
Blue creatuers have been found in underground cavern in Maxico, which are mysteriously filled with peanut butter pockets.
These pockets are rich in proteins and enabled a civilization of these creatures to flurish and 
build technologies which have surpassed human understanding. Scientist have studied these
creataures and found incredible discoveries, including their mastery of the Spanish language and perfect Tacos!
The Blue creatuers are a race of intelligent creatures who have been discovered in the underground caverns of Maxico. They are a species of intelligent beings who have evolved from the blue creatuers of the planet Earth. They have been known to live in the caverns and have been seen to be able to communicate with each other.
The blue creatuers are a very intellig

,id,text,result
0,0,\nBlue creatuers have been found in undergroun...,[ \nBlue creatuers have been found in undergro...


#### Lovecraftian Horror Stories 

In [14]:
lovecraft_text = """
At this horror I sank nearly to the lichened earth, 
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame, 
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected, 
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants, 
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall."""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, lovecraft_text,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
At this horror I sank nearly to the lichened earth, 
transfixed with a dread not of this nor any world, but only of the mad spaces between the stars.

Out of the unimaginable blackness beyond the gangrenous glare of that cold flame, 
out of the Tartarean leagues through which that oily river rolled uncanny, unheard, and unsuspected, 
there flopped rhythmically a horde of tame, trained, hybrid winged things that no sound eye could ever wholly grasp,
or sound brain ever wholly remember. They were not altogether crows, nor moles, nor buzzards, nor ants, 
nor vampire bats, nor decomposed human beings; but something I cannot and must not recall.
I was not alone in my horror. The other creatures were equally terrified.
The first thing I saw was a black, black thing,  with a black f

,id,text,result
0,0,\nAt this horror I sank nearly to the lichened...,[ \nAt this horror I sank nearly to the lichen...


In [15]:
cthulu_intro = """
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. 
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. 
The sciences, each straining in its own direction, have hitherto harmed us little"""

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, cthulu_intro,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The most merciful thing in the world, I think, is the inability of the human mind to correlate all its contents. 
We live on a placid island of ignorance in the midst of black seas of infinity, and it was not meant that we should voyage far. 
The sciences, each straining in its own direction, have hitherto harmed us little.  They have not yet made us aware of the fact that we are not alone in the universe.  
I am not a scientist, but I am a man of science.   
I have been a man for many years, and I have been an artist for many more. 




,id,text,result
0,0,"\nThe most merciful thing in the world, I thin...","[ \nThe most merciful thing in the world, I th..."


#### Re-Write movie scripts

In [16]:
the_matrix_script = """
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(1000)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The Architect: Hello, Neo.

Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  I am here to answer your questions.  You are a human being, and you are here to learn.  

You are here because you are human. You are here for the same reason that I am here.   

I am the architect.  I am your architect.
 You have been given the opportunity to learn, and to grow. 




,id,text,result
0,0,"\nThe Architect: Hello, Neo.\n\nNeo: Who are y...","[ \nThe Architect: Hello, Neo.\n\nNeo: Who are..."


In [17]:
the_matrix_script = """
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect: """

gpt2.setMaxOutputLength(500)
generate_with_gpt2(pipeline, the_matrix_script,task='generate', print=True)

Example 0: ________________________________________________________________________________________________________________________________________________________________________________________________________
 
The Architect: Hello, Neo.
Neo: Who are you?

The Architect: I am the Architect. I created the matrix. I've been waiting for you. 
You have many questions, and although the process has altered your consciousness, you remain irrevocably human. 
Ergo, some of my answers you will understand, and some of them you will not. 
Concordantly, while your first question may be the most pertinent, you may or may not realize it is also irrelevant.

Neo: Why am I here?

The Architect:  I am here to help you.
 You are a human being.  You have a mind.  

You are not a machine.    You are not an android.      You are a living being.
You can be a robot.  You can be an android, or a living human being, or an android and a living robot.
The Matrix is a living, breathing, sentient, living being, 

,id,text,result
0,0,"\nThe Architect: Hello, Neo.\nNeo: Who are you...","[ \nThe Architect: Hello, Neo.\nNeo: Who are y..."
